In [1]:
import logging

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig

In [2]:
import azureml
subscription_id = "57c0a109-5ef9-4713-aaf0-415f299efa76"
resource_group = "AutoMLRG2"
workspace_name = "AutoMLPoc"
workspace_region = "eastus" 
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace
 
ws = Workspace.create(name = workspace_name,                    
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                     exist_ok = True)
ws.get_details()
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)
 
# persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Falling back to use azure cli credentials. This fall back to use azure cli credentials will be removed in the next release. 
Make sure your code doesn't require 'az login' to have happened before using azureml-sdk, except the case when you are specifying AzureCliAuthentication in azureml-sdk.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AUTRAGDJC to authenticate.
Interactive authentication successfully completed.
Wrote the config file config.json to: /databricks/driver/aml_config/config.json

In [3]:
ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-local-classification'
project_folder = './sample_projects/automl-local-classification'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

Found the config file in: /databricks/driver/aml_config/config.json
 Out[ 3 ]: 
 
Experiment Name automl-local-classification 
Location eastus 
Project Directory ./sample_projects/automl-local-classification
Resource Group AutoMLRG2 
SDK version 1.0.15 
Subscription ID 57c0a109-5ef9-4713-aaf0-415f299efa76 
Workspace Name AutoMLPoc

In [4]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on.

In [5]:
import time
import datetime
import sys
import io
from pyspark.sql import *
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.types import DecimalType, StringType, DateType, DoubleType, IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit, col, expr, when, round, concat
from pyspark.sql.functions import from_unixtime, unix_timestamp
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

readConfig = {
  "Endpoint" : "https://automlpoc.documents.azure.com:443/",
  "Masterkey" : "ug9douHKHXAnDEWH2Gwdt1W4MyqB3HzghbTMjTalNIrBYwSiykUrEJMdtm2ZPUt1tGlOa3YMIXsDfG0vjjcHdQ==",
  "Database" : "AutoMlPoc",
  "Collection" : "AutoMLData",
  "query_pagesize" : "2147483647",
  "query_custom" : "select * from AutoMLData where AutoMLData.datatype = 'training'"
}

# Connect via azure-cosmosdb-spark to create Spark DataFrame
flights = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()
data = flights.select('data').collect()
target = flights.select('target').collect()
cosmosdbData = datasets.base.Bunch(data=np.asarray(data[0][0]), target=np.asarray(target[0][0]))
X_train = cosmosdbData.data[100:,:]
y_train = cosmosdbData.target[100:]


In [6]:
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'AUC_weighted',
                             iteration_timeout_minutes = 60,
                             iterations = 20,
                             n_cross_validations = 3,
                             verbosity = logging.INFO,
                             X = X_train, 
                             y = y_train,
                             path = project_folder)

In [7]:
local_run = experiment.submit(automl_config, show_output = True)
local_run

Running on local machine
Parent Run ID: AutoML_0244620b-11b7-4b7f-9747-f2f71829440f
*******************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************************

 ITERATION PIPELINE TRAINFRAC DURATION METRIC BEST
 0 StandardScalerWrapper KNN 1.0000 0:00:12 0.9994 0.9994
 1 RobustScaler LightGBM 1.0000 0:00:17 0.9986 0.9994
 2 RobustScaler LogisticRegression 1.0000 0:00:13 0.9960 0.9994
 3 StandardScalerWrapper KNN 1.0000 0:00:09 0.9988 0.9994
 4 StandardScalerWrapper KNN 1.0000 0:00:11 0.9985 0.9994
 5 MaxAbsScaler LightGBM 1.0000 0:00:12 0.9966 0.9994
 6 StandardScalerWrapper LogisticRegression 1.0000 0:00:09 0.9989 0.9994
 7 RobustScaler LogisticRegression 1.0000 0:00:11 0.9960 0.9994
 8 StandardScalerWrapper LightGBM 1.0000 0:00:10 0.9931 0.9994
 9 StandardScalerWrapper LogisticRegression 1.0000 0:00:16 0.9945 0.9994
 10 StandardScalerWrapper LightGBM 1.0000 0:00:11 0.9907 0.9994
 11 TruncatedSVDWrapper KNN 1.0000 0:00:08 0.9988 0.9994
 12 StandardScalerWrapper LightGBM 1.0000 0:00:14 0.9990 0.9994
 13 MinMaxScaler ExtremeRandomTrees 1.0000 0:00:09 0.9882 0.9994
 14 StandardScalerWrapper SVM 1.0000 0:00:10 0.9995 0.9995
 15 MaxAbsScaler SVM 1.0000 0:00:08 0.9997 0.9997
 16 RobustScaler LightGBM 1.0000 0:00:14 0.9989 0.9997
 17 TruncatedSVDWrapper KNN 1.0000 0:00:08 0.9992 0.9997
 18 RobustScaler DecisionTree 1.0000 0:00:07 0.9459 0.9997
 19 Ensemble 1.0000 0:00:16 0.9997 0.9997
 Out[ 45 ]: 
Run(Experiment: automl-local-classification,
Id: AutoML_0244620b-11b7-4b7f-9747-f2f71829440f,
Type: automl,
Status: Completed)